# Schudeler Query from Silvewr to Gold for Champions

Library

In [48]:
import pandas as pd
from pandas_gbq import read_gbq, to_gbq
import json

Project Info

In [67]:
project_id = "big-data-lol"
dataset_id_from = "Silver_LOL_Dataset"
dataset_id_to = "Gold_LOL_Dataset"
table_id_from_data = "silver_batch_champion"
table_id_from_image = "silver_batch_img_champion"
table_id_to = "gold_batch_champion"

Lol current patch

In [50]:
current_patch = "14.12"

Query

In [61]:
query = f'''
SELECT data.id, data.key, img.image
FROM {project_id}.{dataset_id_from}.{table_id_from_data} as data join
{project_id}.{dataset_id_from}.{table_id_from_image} as img
on data.id = img.key
WHERE TIMESTAMP_TRUNC(data._PARTITIONTIME, DAY) = TIMESTAMP("2024-06-17") and
TIMESTAMP_TRUNC(img._PARTITIONTIME, DAY) = TIMESTAMP("2024-06-17")
'''

Query in BigQuery Dataset

In [62]:
df = read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


Schema create

In [65]:
from google.cloud import bigquery
schema = [

  bigquery.SchemaField('id', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('key', 'INTEGER', 'REQUIRED'),
  bigquery.SchemaField('img', 'STRING', 'REQUIRED'),

]



In [68]:
client = bigquery.Client(project=project_id)

validated_path = '/tmp/validated_' + "champs_to_silver.csv"
# se transforma el df a csv
df.to_csv(validated_path, index=False)

print(validated_path)
# Cargar los datos validados en BigQuery
dataset_ref = client.dataset(dataset_id_to)
table_ref = dataset_ref.table(table_id_to)

#configuracion del job
job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = schema
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


with open(validated_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()


/tmp/validated_champs_to_silver.csv


LoadJob<project=big-data-lol, location=US, id=8062c2c0-c885-45c7-be28-6ee93309bedc>

Job Configuration